In [11]:
%%capture
import pandas as pd
import numpy as np
import os
import re
import gensim
from gensim.models import Word2Vec
import nltk
from nltk import RegexpTokenizer as rpt
from nltk.corpus import stopwords as sw
from nltk import tokenize    
from string import punctuation 

nltk.download('punkt')
nltk.download('stopwords')
stopwords = sw.words('portuguese')
    
def parse(text):
    words = []
    word_pattern = rpt(r'\w+')
    
    patterns = [word_pattern]
    
    for pattern in patterns:
        tokens = []
        for token in pattern.tokenize(text):
            if token not in stopwords and len(token) > 3:
                tokens.append(token)
        words.extend(tokens)
    return words

model = Word2Vec.load('pt.bin') ### carregando modelo pré-treinado
model.init_sims(replace=True) ### normalizando modelo

In [18]:
noticia1 = "doenças comuns no brasil"
noticia2 = "inscrições do enem abrem sexta-feira"

noticia1 = " ".join(parse(noticia1))
noticia2 = " ".join(parse(noticia2))

def get_embeddings(headline, model):
        for word in headline.split():
            return model[word]

noticia1_embedding = get_embeddings(noticia1, model)
noticia2_embedding = get_embeddings(noticia2, model)

print("Tamanho do embedding:\n",noticia1,":",len(noticia1_embedding),"\n",noticia2,":",len(noticia2_embedding))

Tamanho do embedding:
 doenças comuns brasil : 300 
 inscrições enem abrem sexta feira : 300


/home/liraop/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':
